<a href="https://colab.research.google.com/github/m-bashari-m/sample-DNN-projects/blob/main/4_3cat_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [132]:
train_ds = tf.data.Dataset.list_files(str(train_dir+'/*/*'))
val_ds = tf.data.Dataset.list_files(str(validation_dir+'/*/*'))

In [133]:
for path in train_ds.take(5):
  print(path)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.892.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.684.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.765.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.86.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.548.jpg', shape=(), dtype=string)


In [134]:
def prepare_labeled_data(path):
  label = tf.strings.split(path, '/')[-2]
  image = tf.image.decode_image( tf.io.read_file(path), expand_animations=False )
  return image, label

In [135]:
def augmentation(img):
  img = tf.image.resize_with_crop_or_pad(img, 260, 260)
  img = tf.image.random_crop(img, size=[256, 256, 3])
  img = tf.image.random_brightness(img, max_delta=.2)
  img = tf.image.random_flip_left_right(img)
  return img

In [136]:
def normalization(img):
  img = tf.image.convert_image_dtype(img, tf.float32)
  return img / 255

In [137]:
def train_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = augmentation(img)
  img = normalization(img)
  return img, label

In [138]:
def test_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = normalization(img)
  return img, label

In [139]:
BATCH_SIZE=64
AUTOTUNE = tf.data.AUTOTUNE


In [140]:
train_ds = train_ds.map(train_preprocessing).shuffle(AUTOTUNE, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = map(test_preprocessing).batch(BATCH_SIZE).prefetch(AUTOTUNE)

TypeError: ignored

In [ ]:
for img, lbl in train_ds.take(10):
  print(img.shape, lbl.numpy().decode('UTF-8'))